In [1]:
import scipy.io 
import matplotlib.pyplot as plt
import cv2
import keras
from glob import glob
import numpy as np
from tqdm import tqdm
import os

from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


C:\Users\leetw\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
normal_paths = glob('E:/Datasets/PathoBarIlan/Case8/Normal*.mat')
cancer_paths = glob('E:/Datasets/PathoBarIlan/Case8/Cancer*.mat')
mixed_paths = glob('E:/Datasets/PathoBarIlan/Case8/Mixed*.mat')

data_dir = 'E:/Datasets/PathoBarIlan/Case8'

pos_name_init = 'Cancer'
neg_name_init = 'Normal'

use_rgb = True # True=rgb, False=spectral

window_size = (200, 200)
shift = (100, 100)

In [3]:
def read_slide(path):
    mat = scipy.io.loadmat(path)
    spectral = mat["Spec"]
    rgb = mat["Section"]
    shape = rgb.shape
    
    return spectral, rgb

In [4]:
def create_batch_of_crops_from_slide(img, window_size, shift, vis_flag=False):
    crops = []

    n_iter_x = (img.shape[1]-window_size[0])//shift[0] + 1

    n_iter_y = (img.shape[0]-window_size[1])//shift[1] + 1

#     n_iter_x, n_iter_y

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            init_y = i*shift[0]
            init_x = j*shift[1]
        
            crops.append(img[init_x:init_x+window_size[0], init_y:init_y+window_size[1], :])
    if vis_flag:
        visualize_batch_of_crops(crops, n_iter_y, n_iter_x)
    return crops

In [5]:
def visualize_batch_of_crops(crops, n_iter_y, n_iter_x):
    fig, axes = plt.subplots(n_iter_y, n_iter_x, figsize=(5, 5), gridspec_kw = {'wspace':0, 'hspace':0})

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            axes[j, i].imshow(crops[i*n_iter_y + j])
            axes[j, i].axis('off')
            axes[j, i].set_aspect('equal')
    plt.show()

In [6]:
def create_crops_from_fileslist(fileslist, window_size, shift):
    rgb_crops = []
    spectral_crops = []
    labels = []

    for file in tqdm(fileslist):
#         print(file)
        spectral, rgb = read_slide(file)
        spectral_crops += create_batch_of_crops_from_slide(spectral, window_size=window_size, shift=shift)
        added_rgb_crops = create_batch_of_crops_from_slide(rgb, window_size=window_size, shift=shift)
        rgb_crops += added_rgb_crops
        file_name = os.path.basename(file)
        if pos_name_init in file_name:
            labels += [True]*len(added_rgb_crops)
        elif neg_name_init in file_name:
            labels += [False]*len(added_rgb_crops)
        else:
            raise ValueError('File {} is not in the right format ({}-pos, {}-neg)'.format(file_name, pos_name_init, neg_name_init))
#         print(labels)
    out_labels = to_categorical(labels)
    out_sepc = np.stack(spectral_crops)
    out_rgb = np.stack(rgb_crops)
    
    
    return out_sepc, out_rgb, out_labels

In [7]:
def create_crops_from_dir(dir_path, window_size, shift):
    fileslist = glob(dir_path + '/*')
    spectral_crops, rgb_crops, labels = create_crops_from_fileslist(fileslist, window_size, shift)
    return spectral_crops, rgb_crops, labels

## test and vis

## prepare data

In [8]:
train_spectral, train_rgb, train_labels = create_crops_from_dir(data_dir+'/Train', window_size=window_size, shift=shift)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  2.16it/s]


In [9]:
test_spectral, test_rgb, test_labels = create_crops_from_dir(data_dir+'/Test', window_size=window_size, shift=shift)
eval_spectral, eval_rgb, eval_labels = create_crops_from_dir(data_dir+'/Eval', window_size=window_size, shift=shift)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.54it/s]


In [10]:
print(eval_labels, train_labels, test_labels)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]] [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]


##### old prepare data

## build and train model

In [11]:
if use_rgb:
    x_train = train_rgb
    x_eval = eval_rgb
    x_test = test_rgb
else:
    x_train = train_spectral
    x_eval = eval_spectral
    x_test = test_spectral
y_train = train_labels
y_eval = eval_labels
y_test = test_labels

In [13]:
print('Data:')
print(y_train.shape)
train_pos = y_train[:,0].sum()
eval_pos = y_eval[:,0].sum()
test_pos = y_test[:,0].sum()

print('Train: {}/{} (pos/neg)'.format(train_pos, len(y_train)-train_pos ))
print('Test: {}/{} (pos/neg)'.format(test_pos, len(y_test)-test_pos ))
print('Eval: {}/{} (pos/neg)'.format(eval_pos, len(y_eval)-eval_pos ))

Data:
(150, 2)
Train: 50.0/100.0 (pos/neg)
Test: 16.0/16.0 (pos/neg)
Eval: 6.0/3.0 (pos/neg)


In [14]:
mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=x_train[0].shape, classes=2, dropout=0.2)

In [15]:
mobilenet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 202, 202, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 100, 100, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 100, 100, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 100, 100, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 102, 102, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)      288       
__________

_________________________________________________________________
conv_pad_9 (ZeroPadding2D)   (None, 15, 15, 512)       0         
_________________________________________________________________
conv_dw_9 (DepthwiseConv2D)  (None, 13, 13, 512)       4608      
_________________________________________________________________
conv_dw_9_bn (BatchNormaliza (None, 13, 13, 512)       2048      
_________________________________________________________________
conv_dw_9_relu (Activation)  (None, 13, 13, 512)       0         
_________________________________________________________________
conv_pw_9 (Conv2D)           (None, 13, 13, 512)       262144    
_________________________________________________________________
conv_pw_9_bn (BatchNormaliza (None, 13, 13, 512)       2048      
_________________________________________________________________
conv_pw_9_relu (Activation)  (None, 13, 13, 512)       0         
_________________________________________________________________
conv_pad_1

In [16]:
optimizer = Adam(lr=1e-3)
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer)
lrReduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=1, min_lr=1e-6)
chkpnt = ModelCheckpoint("my_models/model_spec", save_best_only=True)

In [17]:
print(x_train.shape, y_train.shape)
print(x_eval.shape, y_eval.shape)

(150, 200, 200, 3) (150, 2)
(9, 200, 200, 3) (9, 2)


In [18]:
mobilenet_model.fit(x=x_train, y=y_train, epochs=1000, validation_data=(x_eval, y_eval), batch_size=64, verbose=2, callbacks=[chkpnt, lrReduce], shuffle=True)

Train on 150 samples, validate on 9 samples
Epoch 1/1000
 - 11s - loss: 0.3424 - val_loss: 5.3434
Epoch 2/1000
 - 2s - loss: 0.0095 - val_loss: 5.3434
Epoch 3/1000
 - 2s - loss: 3.2225e-05 - val_loss: 5.3434
Epoch 4/1000
 - 2s - loss: 1.8803e-05 - val_loss: 5.3434
Epoch 5/1000
 - 2s - loss: 0.0014 - val_loss: 5.3434
Epoch 6/1000
 - 2s - loss: 1.2301e-05 - val_loss: 5.3434

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 7/1000
 - 2s - loss: 3.7384e-06 - val_loss: 5.3434
Epoch 8/1000
 - 2s - loss: 4.4285e-05 - val_loss: 5.3434
Epoch 9/1000
 - 2s - loss: 2.9211e-05 - val_loss: 5.3434
Epoch 10/1000
 - 2s - loss: 7.7385e-06 - val_loss: 5.3434

Epoch 00010: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
Epoch 11/1000
 - 2s - loss: 1.0488e-04 - val_loss: 5.3434
Epoch 12/1000
 - 2s - loss: 1.9728e-05 - val_loss: 5.3434
Epoch 13/1000
 - 2s - loss: 8.7006e-06 - val_loss: 4.8999
Epoch 14/1000
 - 2s - loss: 1.4264e-05 - val_loss: 3.6521
Ep

KeyboardInterrupt: 

In [19]:
y_pred = mobilenet_model.predict(test_rgb)

In [20]:
y_pred = y_pred[:, 0]>y_pred[:, 1]

In [21]:
y_test = y_test[:, 0]>y_test[:, 1]

In [22]:
(y_test.T[0] != y_pred).sum()

16

In [23]:
len(y_test)

32